In [ ]:
#IMPORTS
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import requests
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_sp500_tickers():
    """Guncel S&P 500 Listesini cekelim."""

    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    tables = pd.read_html(response.text)
    df = tables[0]
    
    tickers = df['Symbol'].tolist()
    # Stock Price isimlendirme duzeltmesi
    tickers = [t.replace('.', '-') for t in tickers]
    return tickers

tickers = get_sp500_tickers()
print("",len(tickers), "\n",tickers[:10])

 501 
 ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


In [ ]:
def download_data(start_date, end_date, tickers):
    """
    Belirtilen tarih araliginda Adjusted Close ve Volume verilerini indirelim.
    """
    print(f"{len(tickers)} hisse icin veri indiriliyor...")

    # yfinance ile toplu indirme
    data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', auto_adjust=True)

    prices = data.xs('Close', level=1, axis=1)
    volumes = data.xs('Volume', level=1, axis=1)
    return prices, volumes


# Parametre
START_DATE = "2010-01-01"
END_DATE   = "2024-12-31"

#
tickers = get_sp500_tickers()


prices_df, volumes_df = download_data(START_DATE, END_DATE, tickers)

# 
valid_columns = prices_df.columns[prices_df.notna().sum() > (len(prices_df) * 0.8)]
prices_df = prices_df[valid_columns]
volumes_df = volumes_df[valid_columns]

print("DATA DOWNLOADED")
print(f"Shape of data: {prices_df.shape}")
print(prices_df.head())


# Veriyi kaydetmek istersen:
# prices_df.to_csv("sp500_prices_2010_2024.csv")
# volumes_df.to_csv("sp500_volumes_2010_2024.csv")

501 hisse icin veri indiriliyor...


[**                     4%                       ]  22 of 501 completed$SOLS: possibly delisted; no price data found  (1d 2010-01-01 -> 2024-12-31) (Yahoo error = "Data doesn't exist for startDate = 1262322000, endDate = 1735621200")
[*****************     35%                       ]  177 of 501 completed$Q: possibly delisted; no price data found  (1d 2010-01-01 -> 2024-12-31) (Yahoo error = "Data doesn't exist for startDate = 1262322000, endDate = 1735621200")
[**********************86%****************       ]  429 of 501 completed$SNDK: possibly delisted; no price data found  (1d 2010-01-01 -> 2024-12-31) (Yahoo error = "Data doesn't exist for startDate = 1262322000, endDate = 1735621200")
[*********************100%***********************]  501 of 501 completed

3 Failed downloads:
['SOLS', 'Q', 'SNDK']: possibly delisted; no price data found  (1d 2010-01-01 -> 2024-12-31) (Yahoo error = "Data doesn't exist for startDate = 1262322000, endDate = 1735621200")


DATA DOWNLOADED
Shape of data: (3773, 446)
Ticker      BLDR   INCY        AWK         BG        TRV         PH        NI  \
Date                                                                            
2010-01-04  3.77   9.50  15.696339  43.723572  34.303448  41.591511  3.522926   
2010-01-05  3.59  10.27  15.855726  45.231731  33.490799  41.393158  3.513841   
2010-01-06  3.60  11.00  15.938890  46.699341  33.015602  41.217709  3.472956   
2010-01-07  3.57  10.82  15.973536  47.970802  33.490799  41.705936  3.429797   
2010-01-08  3.54  10.94  15.938890  48.214272  33.442589  42.888378  3.411628   

Ticker           SPGI    ON       PSKY  ...        FRT       MDLZ      EBAY  \
Date                                    ...                                   
2010-01-04  25.785328  8.87  10.735094  ...  38.977909  12.608082  8.952066   
2010-01-05  25.808342  8.79  10.682360  ...  38.931374  13.224010  8.858429   
2010-01-06  25.578192  8.90  10.343354  ...  38.826714  13.315940  8.8022